<table style="width: 100%;">
    <tr style="background-color: transparent;"><td>
        <img src="https://d8a-88.github.io/econ-fa19/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 12pt;"><strong>Economic Models</strong>, Fall 2019<br>
            Dr. Eric Van Dusen</p></td></tr>
</table>

# Lab 11: Environmental Economics

In this lab, we will be exploring a dataset from an electricity strategy game that looks at porfolio of different power plants, companies, and their outputs. We will integrating data science into environmental economics. 

In [1]:
from datascience import *
import pandas as pd
import numpy as np
from ipywidgets import interact, Dropdown, IntSlider
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from utils import *
%matplotlib inline
plt.rcParams["figure.figsize"] = [10,6]

In the cell below, we load in the ESG table and sort it by the total variable cost column, `Total_Var_Cost_USDperMWH`. The result is stored as `ESG_sorted`.

In [2]:
ESG = Table.read_table("ESGPorfolios_forcsv.csv")
ESG_sorted = ESG.sort("Total_Var_Cost_USDperMWH")
ESG_sorted

Group,Group_num,UNIT NAME,Capacity_MW,Heat_Rate_MMBTUperMWh,Fuel_Price_USDperMMBTU,Fuel_Cost_USDperMWH,Var_OandM_USDperMWH,Total_Var_Cost_USDperMWH,Carbon_tonsperMWH,FixedCst_OandM_perDay,Plant_ID
Old Timers,7,BIG CREEK,1000,nan,0,0,0,0,0,"$15,000",61
Fossil Light,8,HELMS,800,nan,0,0,0.5,0.5,0,"$15,000",72
Fossil Light,8,DIABLO CANYON 1,1000,1,7.5,7.5,4,11.5,0,"$20,000",75
Bay Views,4,MOSS LANDING 6,750,6.9,4.5,31.06,1.5,32.56,0.37,"$8,000",33
Bay Views,4,MOSS LANDING 7,750,6.9,4.5,31.06,1.5,32.56,0.37,"$8,000",34
Old Timers,7,MOHAVE 1,750,10,3,30,4.5,34.5,0.94,"$15,000",62
Old Timers,7,MOHAVE 2,750,10,3,30,4.5,34.5,0.94,"$15,000",63
Big Coal,1,FOUR CORNERS,1900,11.67,3,35,1.5,36.5,1.1,"$8,000",11
Bay Views,4,MORRO BAY 3&4,665,8.02,4.5,36.11,0.5,36.61,0.43,"$4,000",32
East Bay,6,PITTSBURGH 5&6,650,8.02,4.5,36.11,0.5,36.61,0.43,"$2,500",52


Now that we have sorted our table by the "Total_Var_Cost_USDperMWH," we want to see the distribution of Capacity_MW and Variable Cost for a particular group. For a given capacity, what is the variable cost, or cost of producing an additional unit of capacity? 

From the ESG_sorted table above, we are grouping by the first column, Group, to look closer at a particular energy source. 

Select a particular group in the dropdown below to see a barchart of Variable Cost vs. Capacity_MW.

In [3]:
interact(lambda selection: plot_group(selection, ESG_sorted), selection=Dropdown(options=np.unique(ESG["Group"])));

/srv/app/venv/lib/python3.6/site-packages/traitlets/traitlets.py:567: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


interactive(children=(Dropdown(description='selection', options=('Bay Views', 'Beachfront', 'Big Coal', 'Big G…

Suppose we are not concerned with one particular group, and we want to see the distribution of Variable Cost for Capacity_MW across all energy sources. Each color shows a unique group below.

How do Capacity and Variable Cost affect each other? In order to determine the market price of energy in our simulation, demand is an important factor. We don't know exactly how much energy will be demanded in a given frame of time, however we can make predictions based off of estimates that we are given, and use those predictions to calculate the profitability of our plants.

We calculate the maximum variable cost companies can have in order to make profit based on an estimated demand. For now, we will make the assumption that plants are willing to sell at a price equal to their variable cost.

In the graph below, we added the vertical line for demand and horizontal line for variable cost cap. Since we have our power plants in order of lowest variable cost to highest variable cost, we can see that the companies to the left of the vertical demand line will produce energy while the companies to the right of the vertical demand line will choose not to. 

Why? The market will purchase energy from the power plants that have the cheapest prices, and we have graphed the cumulative energy production of companies ordered by increasing variable cost of production.

Adjust the slider for demand and notice how the lines, price, and Capacity_MW are changing as a result.

In [4]:
interact(lambda demand: all_groups_with_demand(demand, ESG_sorted), 
         demand=IntSlider(value=20000, min=0, max=23000, step=1000));

interactive(children=(IntSlider(value=20000, description='demand', max=23000, step=1000), Output()), _dom_clas…

Now that we have seen how Capacity_MV, Variable Cost, and price change as demand changes, we want to be able to calculate profit generated by a given power plant. 

Run the profit function and the interactive slider in the next two cells below to see how profit changes with demand for each power plant.

In [9]:
def profit(sorted_table, price):
    capacity_subset = sum(sorted_table.where("Total_Var_Cost_USDperMWH", are.below(price))["Capacity_MW"])
    revenue = capacity_subset * price
    cost = 0
    for i in range(len(sorted_table.where("Total_Var_Cost_USDperMWH", are.below(price))["Total_Var_Cost_USDperMWH"])):
        cost += sorted_table.where("Total_Var_Cost_USDperMWH", are.below(price))["Total_Var_Cost_USDperMWH"][i]\
        * sorted_table.where("Total_Var_Cost_USDperMWH", are.below(price))["Capacity_MW"][i]
    return revenue - cost

In [10]:
interact(lambda selection, demand: calc_profit(selection, demand, ESG_sorted), 
         selection=Dropdown(options=np.unique(ESG_sorted["Group"])), 
         demand=IntSlider(value=20000, min=0, max=23000, step=1000));

interactive(children=(Dropdown(description='selection', options=('Bay Views', 'Beachfront', 'Big Coal', 'Big G…

Below, we load in the `bids` table with data on energy price bids for different plants and hours.

In [11]:
bids = Table.read_table("S1_bids_0.csv")
bids

TEAM,TEAM_ID,PORTFOLIO,PORTFOLIO_ID,PLANT,PLANT_ID,PERIOD,PRICE1,PRICE2,PRICE3,PRICE4
Arrow,1,Big_Coal,1,Four Corners,11,1,45,47,40,40
Arrow,1,Big_Coal,1,ALAMITOS_7,12,1,73.72,73.72,73.72,73.72
Arrow,1,Big_Coal,1,HUNTINGTON_BEACH_1-2,13,1,50,52,42,43
Arrow,1,Big_Coal,1,HUNTINGTON_BEACH_5,14,1,66.5,66.5,66.5,66.5
Arrow,1,Big_Coal,1,REDONDO_5-6,15,1,51,53,44,43
Arrow,1,Big_Coal,1,REDONDO_7-8,16,1,48,50,44,43
Becker,2,Big_Gas,2,EL_SEGUNDO_1-2,21,1,44.83,44.83,56,44.83
Becker,2,Big_Gas,2,EL_SEGUNDO_3-4,22,1,41.22,42.2,56,41.67
Becker,2,Big_Gas,2,LONG_BEACH,23,1,52.5,52.5,56,52.5
Becker,2,Big_Gas,2,NORTH_ISLAND,24,1,65.5,65.5,65.5,65.5


With a new bids table with data on energy price bids across different power plants and hours, we will now calculate and visualize the profit our power plants have generated in a given hour. We will also be able to calculate cumulative profits, emissions, and expected emissions.

We will need both of these tables, bids and ESG_sorted, to generate an analysis, so in the following cell we first join the tables based on the column Plant_ID.

In the visualization below, we see how price varies with Capacity for a given hour for all power plants from above as demand changes. 

In [21]:
# make sorted joined table
hour = 2
sorted_joined_table = bids.join("PLANT_ID", ESG_sorted, "Plant_ID").sort("PRICE" + str(hour))
sorted_joined_table

PLANT_ID,TEAM,TEAM_ID,PORTFOLIO,PORTFOLIO_ID,PLANT,PERIOD,PRICE1,PRICE2,PRICE3,PRICE4,Group,Group_num,UNIT NAME,Capacity_MW,Heat_Rate_MMBTUperMWh,Fuel_Price_USDperMMBTU,Fuel_Cost_USDperMWH,Var_OandM_USDperMWH,Total_Var_Cost_USDperMWH,Carbon_tonsperMWH,FixedCst_OandM_perDay
61,Heckman,6,Old_Timers,6,BIG_CREEK,1,0.49,0.49,0.49,0.49,Old Timers,7,BIG CREEK,1000,nan,0,0,0,0,0,"$15,000"
72,Krugman,7,Low_Fossil,7,HELMS,1,37,0.5,0.5,37,Fossil Light,8,HELMS,800,nan,0,0,0.5,0.5,0,"$15,000"
75,Krugman,7,Low_Fossil,7,DIABLO_CANYON_1,1,37.5,11.5,11.5,37.5,Fossil Light,8,DIABLO CANYON 1,1000,1,7.5,7.5,4,11.5,0,"$20,000"
62,Heckman,6,Old_Timers,6,MOHAVE_1,1,36.49,36.49,36.49,36.49,Old Timers,7,MOHAVE 1,750,10,3,30,4.5,34.5,0.94,"$15,000"
63,Heckman,6,Old_Timers,6,MOHAVE_2,1,36.49,36.49,36.49,36.49,Old Timers,7,MOHAVE 2,750,10,3,30,4.5,34.5,0.94,"$15,000"
22,Becker,2,Big_Gas,2,EL_SEGUNDO_3-4,1,41.22,42.2,56,41.67,Big Gas,2,EL SEGUNDO 3&4,650,8.83,4.5,39.72,1.5,41.22,0.47,"$1,000"
25,Becker,2,Big_Gas,2,ENCINA,1,41.67,42.2,58,41.67,Big Gas,2,ENCINA,950,9.15,4.5,41.17,0.5,41.67,0.49,"$2,000"
51,Friedman,5,East_Bay,5,PITTSBURGH_1-4,1,50,42.38,58.27,43,East Bay,6,PITTSBURGH 1-4,650,8.99,4.5,40.44,0.5,40.94,0.48,"$2,500"
52,Friedman,5,East_Bay,5,PITTSBURGH_5-6,1,40,42.38,58.27,41,East Bay,6,PITTSBURGH 5&6,650,8.02,4.5,36.11,0.5,36.61,0.43,"$2,500"
55,Friedman,5,East_Bay,5,CONTRA_COSTA_6-7,1,50,42.38,58.27,43,East Bay,6,CONTRA COSTA 6&7,700,8.67,4.5,39,0.5,39.5,0.46,"$6,000"


In [19]:
interact(lambda demand, hour: all_group_bids(demand, hour, sorted_joined_table), 
         demand=IntSlider(value=20000, min=0, max=23000, step=1000), hour=Dropdown(options=[1,2,3,4]));

interactive(children=(IntSlider(value=20000, description='demand', max=23000, step=1000), Dropdown(description…

Now, let's examine a particular power plant for a given hour. 

In [22]:
interact(lambda selection, hour, demand: your_portfolio_plot(selection, hour, demand, sorted_joined_table), 
         selection=Dropdown(options=np.unique(ESG_sorted["Group"])), hour=Dropdown(options=[1,2,3,4]), 
         demand=IntSlider(value=20000, min=0, max=23000, step=1000));

interactive(children=(Dropdown(description='selection', options=('Bay Views', 'Beachfront', 'Big Coal', 'Big G…

Given a power plant, hour, and demand, we have the market price. Below, we explore how we compare market price of energy for a given hour to the marginal costs of running the power plant plants that have a bid price that was below the market price. 

In [20]:
interact(lambda selection, hour, demand: marginal_cost_plot(selection, hour, demand, sorted_joined_table), 
         selection=Dropdown(options=np.unique(ESG_sorted["Group"])), hour=Dropdown(options=[1,2,3,4]), 
         demand=IntSlider(value=20000, min=0, max=23000, step=1000));

interactive(children=(Dropdown(description='selection', options=('Bay Views', 'Beachfront', 'Big Coal', 'Big G…

In [24]:
interact(lambda selection, hour, demand: calc_profit_by_hour(selection, hour, demand, sorted_joined_table), 
         selection=Dropdown(options=np.unique(ESG_sorted["Group"])), hour=Dropdown(options=[1,2,3,4]), 
         demand=IntSlider(value=20000, min=0, max=23000, step=1000));

interactive(children=(Dropdown(description='selection', options=('Bay Views', 'Beachfront', 'Big Coal', 'Big G…